In [34]:
import os
import math
import json

import numpy as np
import torch as t
from pathlib import Path
from pyro.infer.mcmc import NUTS, HMC
from pyro.infer.mcmc.api import MCMC
from sacred import Experiment
from sacred.utils import apply_backspaces_and_linefeeds
from sacred.observers import FileStorageObserver

from bnn_priors.data import UCI, CIFAR10
from bnn_priors.models import RaoBDenseNet, DenseNet, PreActResNet18, PreActResNet34
from bnn_priors.prior import LogNormal
from bnn_priors import prior
from bnn_priors.inference import SGLDRunner

In [3]:
ex = Experiment("bnn_evaluation", interactive=True)

In [39]:
config_file = "../logs/10/config.json"

In [40]:
ex.add_config(config_file)

In [41]:
log_dir = os.path.dirname(config_file)

eval_dir = os.path.join(log_dir, "eval")

os.makedirs(eval_dir, exist_ok=True)

In [42]:
runfile = os.path.join(log_dir, "run.json")

with open(runfile) as infile:
    run_data = json.load(infile)

run_data["artifacts"]

assert "samples.pt" in run_data["artifacts"], "No samples found"

assert "bn_params.pt" in run_data["artifacts"], "No bn_params found"

samples = t.load(os.path.join(log_dir, "samples.pt"))

bn_params = t.load(os.path.join(log_dir, "bn_params.pt"))

In [54]:
dir(ex)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_named_config',
 '_check_command',
 '_create_config_dict',
 '_create_run',
 '_handle_help',
 '_is_traversing',
 'add_artifact',
 'add_config',
 'add_named_config',
 'add_package_dependency',
 'add_resource',
 'add_source_file',
 'additional_cli_options',
 'additional_host_info',
 'all_cli_options',
 'automain',
 'base_dir',
 'capture',
 'captured_functions',
 'captured_out_filter',
 'command',
 'commands',
 'config',
 'config_hook',
 'config_hooks',
 'configurations',
 'current_run',
 'default_command',
 'dependencies',
 'doc',
 'gather_commands',
 'gather_named_configs',
 'get_default_options',
 'get_experiment_info

In [60]:
ex.configurations[2]._conf

{'batch_size': None,
 'batchnorm': True,
 'bias_loc': 0.0,
 'bias_prior': 'gaussian',
 'bias_scale': 1.0,
 'burnin': 10,
 'cycles': 1,
 'data': 'UCI_boston',
 'device': 'try_cuda',
 'inference': 'SGLD',
 'lr': 0.0005,
 'model': 'densenet',
 'momentum': 0.9,
 'n_samples': 5,
 'precond_update': None,
 'save_samples': True,
 'seed': 155011633,
 'skip': 1,
 'temperature': 1.0,
 'warmup': 10,
 'weight_loc': 0.0,
 'weight_prior': 'gaussian',
 'weight_scale': 1.4142135623730951,
 'width': 50}